# MARKET 

In [1]:
MONEY="ARS"
CRYPT="BTC"
URL="https://www.buda.com/api/v2/markets/"+CRYPT.lower()+"-"+MONEY.lower()+"/order_book.json"
minVolumeTrade=0.0001

# REQUIREMENTS

In [2]:
# !pip install -r requirements.txt
# !pip install trading_api_wrappers

# LIBRARIES

In [3]:
import sys
import ast
import pytz
import time
import hmac
import json
import pickle
import base64
import smtplib
import requests
import numpy as np
import pandas as pd
import configparser
import requests.auth
from datetime import datetime
from email.mime.text import MIMEText
from trading_api_wrappers import Buda
from email.mime.multipart import MIMEMultipart

# FUNCTIONS

In [4]:
#################### BUDA CLASS

class BudaHMACAuth(requests.auth.AuthBase):
    """Adjunta la autenticación HMAC de Buda al objeto Request."""

    def __init__(self, api_key: str, secret: str):
        self.api_key = api_key
        self.secret = secret

    def get_nonce(self) -> str:
        # 1. Generar un nonce (timestamp en microsegundos)
        return str(int(time.time() * 1e6))

    def sign(self, r, nonce: str) -> str:
        # 2. Preparar string para firmar
        components = [r.method, r.path_url]
        if r.body:
            encoded_body = base64.b64encode(r.body).decode()
            components.append(encoded_body)
        components.append(nonce)
        msg = ' '.join(components)
        # 3. Obtener la firma
        h = hmac.new(key=self.secret.encode(),
                        msg=msg.encode(),
                        digestmod='sha384')
        signature = h.hexdigest()
        return signature

    def __call__(self, r):
        nonce = self.get_nonce()
        signature = self.sign(r, nonce)
        # 4. Adjuntar API-KEY, nonce y firma al header del request
        r.headers['X-SBTC-APIKEY'] = self.api_key
        r.headers['X-SBTC-NONCE'] = nonce
        r.headers['X-SBTC-SIGNATURE'] = signature
        return r

#################### READ CONFIG

def get_config(category, key):
    return config[category][key]

#################### Global variables

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig():
    
    global config
    global MONEY
    global CRYPT
    
    config = configparser.ConfigParser()
    config.sections()
    config.read('config'+'_'+MONEY.lower()+'_'+CRYPT.lower()+'.ini')

#################### Creates connection to Buda

def create_connection_buda():
    global API_KEY
    global API_SECRET
    return Buda.Auth(API_KEY, API_SECRET)

#################### Creates insert statement

def get_insert_statement():
    return "INSERT INTO public.ord_hist_spr(id, platform, \"position\", date_open, date_close, price, quantity_initial, quantity_traded, currency_pair, paid_fee) VALUES ({}, '{}', '{}', '{}', clock_timestamp(), {}, {}, {}, '{}', {});"

#################### Alert owners

def enviar_alerta(subject, msg, owners):

    me = get_config('owner_information','ME')
    password = get_config('owner_information','PASSWORD')

    #___Create email object
    
    email = MIMEMultipart('alternative')
    
    #___Set parameters
    
    email['Subject'] = subject
    email['From'] = me
    body = MIMEText('<html><body><p>{}</p></body></html>'.format(msg), 'html')

    #___Attach body
    
    email.attach(body)

    #___Create connection
    
    with smtplib.SMTP_SSL('smtp.gmail.com') as connection:

        connection.login(me, password)
        
        #___Send email
        
        for owner in owners:
            email['To'] = owner
            connection.sendmail(me, owner, email.as_string())
            
#################### SEND DAILY MESSAGE

def daily_email():
    global balanceMON
    global balanceCRY
    global hourCounter
    if (float(datetime.now().hour) == 19.0):
        hourCounter=1
    if (float(datetime.now().hour) == 20.0) and hourCounter==1:
        getCRYinAccount()
        getMONinAccount()
        total_en_cuenta  = balanceMON + ( balanceCRY * sellAmountACUM(balanceCRY))
        subject = "SpreadNet: Daily Update"
        msg = "My Money: <b>${}</b><br><br>My Crypt: <b>{}</b><br><br>Total Account: <b>${}</b>".format(round(getMONinAccount(),2),round(getCRYinAccount(),4),round(total_en_cuenta,2))
        owners = json.loads(config.get('owner_information','OWNERS_TRADE'))
        enviar_alerta(subject, msg, owners)
        hourCounter=0
        
################### CREAR DATABASE DE ORDENES DESDE CERO

def crearDatabaseDesdeCero():
    
    global database_past_asks_bids
    global client
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    database_past_asks_bids=pd.DataFrame()
    time.sleep(round(wait/10))
    pages=int(client.order_pages(market_id=CRYPT+"-"+MONEY)[1].total_pages)
    for page in range(1,pages+1):
        time.sleep(round(wait/10))
        data=pd.DataFrame(client.order_pages(market_id=CRYPT+"-"+MONEY,page=page)[0])
        database_past_asks_bids=database_past_asks_bids.append(data)
    database_past_asks_bids.reset_index(drop=True,inplace=True)
    database_past_asks_bids=database_past_asks_bids.loc[~database_past_asks_bids.limit.isna()]
    database_past_asks_bids.columns = database_past_asks_bids.columns.get_level_values(0)
    database_past_asks_bids.reset_index(drop=True,inplace=True)

    if len(database_past_asks_bids)>0:
        for i in range(0,len(database_past_asks_bids)):
            database_past_asks_bids.at[i,"amount_2"]=database_past_asks_bids.at[i,"amount"][0]
            database_past_asks_bids.at[i,"limit_2"]=database_past_asks_bids.at[i,"limit"][0]
            database_past_asks_bids.at[i,"original_amount_2"]=database_past_asks_bids.at[i,"original_amount"][0]
            database_past_asks_bids.at[i,"paid_fee_2"]=database_past_asks_bids.at[i,"paid_fee"][0]
            database_past_asks_bids.at[i,"total_exchanged_2"]=database_past_asks_bids.at[i,"total_exchanged"][0]
            database_past_asks_bids.at[i,"traded_amount_2"]=database_past_asks_bids.at[i,"traded_amount"][0]

        database_past_asks_bids.drop(["amount","limit","original_amount","paid_fee","total_exchanged","traded_amount"],axis=1,inplace=True)
        new_columns=['id','account_id','created_at','fee_currency','market_id','price_type','state','type','json','amount','limit','original_amount','paid_fee','total_exchanged','traded_amount']
        database_past_asks_bids.columns=new_columns
        database_past_asks_bids=database_past_asks_bids[["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]]

    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
    pickle.dump(database_past_asks_bids, pickle_out)
    pickle_out.close()
    
################## FUNCIÓN QUE CANCELA CUANDO HAY MÁS DE UNA ORDEN

import os
import surbtc

def finishThem():

    global client
    global bidOrderId
    global bidOrderDetails
    global last_volume_traded_fee
    global askOrderId
    global askOrderDetails
    global database_past_asks_bids
    global theorySellExecuted
    global theoryBuyExecuted
    global theorySellPrice
    global theoryBuyPrice
    global API_SECRET
    global API_KEY
    global MONEY
    global CRYPT
    global wait
    global waitException

    askPendings=0
    bidPendings=0
    
    while True:
        try:
            client_surbtc = surbtc.Client(API_KEY,API_SECRET)
            Market = client_surbtc.getMarket(CRYPT+"-"+MONEY)
            data=pd.DataFrame(Market.getPendingOrders())
            break
        except:
#             #printOutput("Exception finishThem():",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            time.sleep(waitException)
    
    if len(data)>0:
    
        askPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")])
        bidPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")])

        if (askPendings>1) or (bidPendings>1):

            askOrderId=None
            askOrderDetails=None
            askIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")].id.values)

            bidOrderId=None
            bidOrderDetails=None
            bidIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")].id.values)

            if len(askIdList)>=1:

                for i in askIdList:
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            askOrderDetailsFinish = client.order_details(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (askOrderDetailsFinish.traded_amount.amount > 0.0):
                        theorySellExecuted=theorySellPrice
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                askOrderDetailsFinish = client.order_details(i)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (askOrderDetailsFinish.state=="canceled") or (askOrderDetailsFinish.state=="traded"):
                            break
                        else: 
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(i)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(askOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

            if len(bidIdList)>=1:

                for j in bidIdList:
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            bidOrderDetailsFinish = client.order_details(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (bidOrderDetailsFinish.traded_amount.amount > 0.0):
                        theoryBuyExecuted=theoryBuyPrice
                        last_volume_traded_fee=float(bidOrderDetailsFinish.paid_fee[0])+last_volume_traded_fee
                        createMarketBidFee()
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                bidOrderDetailsFinish = client.order_details(j)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (bidOrderDetailsFinish.state=="canceled") or (bidOrderDetailsFinish.state=="traded"):
                            break
                        else: 
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(j)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(bidOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

        balancing_Ask_Bid()
#         if askPendings>1:
#             #printOutput("Finish Them [Asks] = ",askPendings)
#         if bidPendings>1:
#             #printOutput("Finish Them [Bids] = ",bidPendings)
            
################################################      

def finishThem_before_run():

    global client
    global bidOrderId
    global bidOrderDetails
    global last_volume_traded_fee
    global askOrderId
    global askOrderDetails
    global database_past_asks_bids
    global theorySellExecuted
    global theoryBuyExecuted
    global theorySellPrice
    global theoryBuyPrice
    global API_SECRET
    global API_KEY
    global MONEY
    global CRYPT
    global wait
    global waitException

    askPendings=0
    bidPendings=0
    
    client_surbtc = surbtc.Client(API_KEY,API_SECRET)
    Market = client_surbtc.getMarket(CRYPT+"-"+MONEY)
    data=pd.DataFrame(Market.getPendingOrders())
    
    if len(data)>0:
    
        askPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")])
        bidPendings=len(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")])

        if (askPendings>=1) or (bidPendings>=1):

            askOrderId=None
            askOrderDetails=None
            askIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Ask")].id.values)

            bidOrderId=None
            bidOrderDetails=None
            bidIdList=list(data.loc[((data.state=="pending") | (data.state=="accepted")) & (data.type=="Bid")].id.values)

            if len(askIdList)>=1:

                for i in askIdList:

                    time.sleep(round(wait/10))
                    while True:
                        try:
                            askOrderDetailsFinish = client.order_details(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (askOrderDetailsFinish.traded_amount.amount > 0.0):
                        theorySellExecuted=theorySellPrice
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(i)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                askOrderDetailsFinish = client.order_details(i)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (askOrderDetailsFinish.state=="canceled") or (askOrderDetailsFinish.state=="traded"):
                            break
                        else:
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(i)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(askOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

            if len(bidIdList)>=1:

                for j in bidIdList:

                    time.sleep(round(wait/10))
                    while True:
                        try:
                            bidOrderDetailsFinish = client.order_details(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (bidOrderDetailsFinish.traded_amount.amount > 0.0):
                        theoryBuyExecuted=theoryBuyPrice
                        last_volume_traded_fee=float(bidOrderDetailsFinish.paid_fee[0])+last_volume_traded_fee
                        createMarketBidFee()
                        write_buy_sell_prices()
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            client.cancel_order(j)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    while(True):
                        time.sleep(wait)
                        while True:
                            try:
                                bidOrderDetailsFinish = client.order_details(j)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                        if (bidOrderDetailsFinish.state=="canceled") or (bidOrderDetailsFinish.state=="traded"):
                            break
                        else:
                            time.sleep(wait/2)
                            while True:
                                try:
                                    client.cancel_order(j)
                                    break
                                except:
                                    time.sleep(waitException)
                                    #printOutput(f'client error waiting {waitException} seconds')

                    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
                    database_past_asks_bids= pickle.load(pickle_in)

                    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
                    appen_order_dataframe=pd.DataFrame(columns=columns)
                    order_dataframe=pd.DataFrame(bidOrderDetailsFinish[14]).head(1)
                    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
                    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
                    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
                    database_past_asks_bids.reset_index(drop=True,inplace=True)

                    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
                    pickle.dump(database_past_asks_bids, pickle_out)
                    pickle_out.close()

        balancing_Ask_Bid()
#         if askPendings>1:
#             #printOutput("Finish Them [Asks] = ",askPendings)
#         if bidPendings>1:
#             #printOutput("Finish Them [Bids] = ",bidPendings)

#################### Request order book
            
def request_order_book():
    global URL
    global API_KEY
    global API_SECRET
    global wait
    global waitException
    
    while True:
        try:
            with requests.get(URL, auth=BudaHMACAuth(API_KEY, API_SECRET)) as r:
                order_book = r.json()
                if ( order_book != None and 'order_book' in order_book ):
                    return order_book['order_book']
                    break
                #else:
                    #printOutput('Order book no llega completo')
        except:
            time.sleep(waitException)
            #printOutput('Order book no llega completo')
            
#################### History trades

def history_trades():
    global pastAsks
    global pastBids
    global sellPrice
    global buyPrice
    global MONEY
    global CRYPT
    
    global dia_de_balanceo
    global contador_balanceos
    global prendido_rebalanceo
    
    updatePast_Asks_Bids()
    if abs(pastAsks-pastBids)<minVolumeTrade:
        sellPrice=0.0
        buyPrice=0.0
        time.sleep(round(wait/10))
        with requests.get( "https://www.buda.com/api/v2/markets/"+CRYPT.lower()+"-"+MONEY.lower()+"/trades.json", auth=BudaHMACAuth(API_KEY, API_SECRET) ) as r:
            history_trades = r.json()

            if ( history_trades != None and 'trades' in history_trades ):
                history_trades=pd.DataFrame(history_trades["trades"]["entries"],columns=["ID1","VOLUME","PRICE","TYPE","ID2"])  
                history_trades.drop(["ID1","ID2"],inplace=True,axis=1)
                history_trades.VOLUME=history_trades.VOLUME.astype("float")
                history_trades_ask=history_trades.loc[history_trades.TYPE=="sell"]
                history_trades_bid=history_trades.loc[history_trades.TYPE=="buy"]
                history_trades_ask.reset_index(drop=True,inplace=True)
                history_trades_bid.reset_index(drop=True,inplace=True)

                recomendedVolume=np.average(list(history_trades.VOLUME.values))

        #_______REBALANCEAR UNA VEZ AL MES
        
#         if int(datetime.now().day)==dia_de_balanceo+1:
#             contador_balanceos=0

#         if (int(datetime.now().day)==dia_de_balanceo) and (contador_balanceos==0):
#             prendido_rebalanceo=1

#         if (prendido_rebalanceo==1) and (contador_balanceos==0):
#             monthlyBalance()
        
        #______
    
        myActualMoney=getMONinAccount()
        myActualCrypt=getCRYinAccount()

#         recomendedVolume=myActualCrypt*0.95
        
        if (recomendedVolume>myActualCrypt*0.95):
            recomendedVolume=myActualCrypt*0.95


        #_____SIMULAR MARKET ORDER CON LA RECOMENDED VOLUME QUE TENGO

        buyingBook=request_order_book()["asks"].copy()
        buyAmount=0.0
        buyPrice=0.0
        buyCounter=0

        buyPriceACUM=0.0
        buyPriceACUM_Theory=0.0
        buyAmountACUM=0.0
        myDifference=0.0

        #_____

        while True:

            buyAmount=float(buyingBook[buyCounter][1])
            buyPrice=float(buyingBook[buyCounter][0])
            buyPriceACUM_Theory=buyPriceACUM_Theory+(buyPrice*buyAmount)

            if buyPriceACUM_Theory>myActualMoney:
                myDifference=(myActualMoney-buyPriceACUM)
                buyAmountACUM=buyAmountACUM+(myDifference/buyPrice)
                break
            else:
                buyPriceACUM=buyPriceACUM+(buyAmount*buyPrice)
                buyAmountACUM=buyAmountACUM+buyAmount
                buyCounter=buyCounter+1

        #_____ 

        if recomendedVolume>buyAmountACUM*0.95:
            recomendedVolume=buyAmountACUM*0.95
                
    else:
        recomendedVolume=abs(pastAsks-pastBids)
    return round(recomendedVolume,5)

#################### Update Limits (Price & Volume)

def updateLimits():
    
    global limitAskPrice
    global limitBidPrice
    global limitAskVolume
    global limitBidVolume
    
    order_book = request_order_book()
    
    limitAskPrice = float(order_book['asks'][0][0])
    limitBidPrice = float(order_book['bids'][0][0])
    limitAskVolume = float(order_book['asks'][0][1])
    limitBidVolume = float(order_book['bids'][0][1])

#################### Update limit prices

def updatePrice():
    order_book = request_order_book()
    return float(order_book['asks'][0][0]), float(order_book['bids'][0][0])
    
#################### Update limit volumes
    
def updateVolume():
    order_book = request_order_book()
    return float(order_book['asks'][0][1]), float(order_book['bids'][0][1])
    
#################### Validate margin for inversion
    
def validMargin(limitAskPrice, limitBidPrice):
    global margin
    global commission
    
    commission=float(readText("trade_commission.txt"))
    #return (limitAskPrice-(commission*(limitAskPrice+limitBidPrice)))/limitBidPrice - 1 > margin
    return (limitAskPrice*(1-2*commission))/limitBidPrice-1 > margin
    
#################### Get CRYPT in account

def getCRYinAccount():
    global client
    global balanceCRY
    global CRYPT
    global wait
    global waitException
    time.sleep(round(wait/10))
    while True:
        try:   
            balance = client.balance(CRYPT)
            balanceCRY = balance.amount.amount   
            break
        except: 
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    return balance.amount.amount

#################### Get MON in account

def getMONinAccount():
    global client
    global balanceMON
    global MONEY
    global wait
    global waitException
    time.sleep(round(wait/10))
    while True:
        try:
            balance = client.balance(MONEY)
            balanceMON = balance.amount.amount    
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    return balance.amount.amount

#################### Create askOrder

def createAsk(limitAskPrice):
    
    global balanceCRY
    global areCRY
    global askOrderId
    global askOrderDetails
    global client
    global askVolume
    global bidVolume
    global pastAsks
    global pastBids
    global theorySellPrice
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    while askOrderId!=None:
        try:
            cancelAsk()
        except:
            pass
            time.sleep(wait/2)
    getCRYinAccount()
    amountCRY = balanceCRY
    if round(askVolume,5) >= minVolumeTrade:
        #printOutput("[AskVolume, BidVolume]: =",askVolume,bidVolume)
        if (amountCRY >= askVolume):
            areCRY = True
            amountAsk = askVolume
            time.sleep(round(wait/10))
            while True:
                try:
                    orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "ask", "limit", amountAsk, limitAskPrice)
                    askOrderId = orden.id
                    askOrderDetails = client.order_details(askOrderId)
                    theorySellPrice=limitAskPrice
                    #printOutput("New Ask Order [Volume,Price,ID] = ",amountAsk,limitAskPrice,askOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
        else:
            areCRY = False
            #printOutput("Stopped: No CRYPT --- createAsk(limitAskPrice)")
            #printOutput(errorCreateAsk)

#################### Create bidOrder

def createBid(limitBidPrice):
    
    global balanceMON
    global areMON
    global bidOrderId
    global bidOrderDetails
    global client
    global askVolume
    global bidVolume
    global pastAsks
    global pastBids
    global theoryBuyPrice
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    while bidOrderId!=None:
        try:
            cancelBid()
        except:
            pass
            time.sleep(wait/2)
    getMONinAccount()
    amountMON = balanceMON
    if round(bidVolume,5) >= minVolumeTrade:
        #printOutput("[AskVolume, BidVolume]: =",askVolume,bidVolume)
        if (amountMON >= limitBidPrice*bidVolume):
            areMON = True
            amountBid = bidVolume
            time.sleep(round(wait/10))
            while True:
                try:
                    orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "bid", "limit", amountBid, limitBidPrice )
                    bidOrderId = orden.id
                    bidOrderDetails = client.order_details(bidOrderId)
                    theoryBuyPrice=limitBidPrice
                    #printOutput("New Bid Order [Volume,Price,ID]= ",amountBid,limitBidPrice,bidOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
        else:
            areMON = False
            #printOutput("Stopped: No MONEY --- createBid(limitBidPrice)")
            #printOutput(errorCreateBid)
            
#################### Create MARKET bidOrder to balance fee comission

def createMarketBidFee():
    
    global last_volume_traded_fee
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    #printOutput("[Checking Fee] :",round(last_volume_traded_fee,5))
    if round(last_volume_traded_fee,5) >= minVolumeTrade:
        time.sleep(round(wait/10))
        while True:
            try:
                orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "bid", "market", round(last_volume_traded_fee,5))
                #printOutput("[MarketBid,Fee]: =",round(last_volume_traded_fee,5))
                last_volume_traded_fee=0.0
                write_buy_sell_prices()
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')

#################### Update askOrder details
        
def updateAskDetails():
    global client
    global askOrderDetails
    global askOrderId
    global wait
    global waitException
    
    if (askOrderDetails != None):
        time.sleep(round(wait/10))
        while True:
            try:
                askOrderDetails = client.order_details(askOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')
        
#################### Update bidOrder details

def updateBidDetails():
    global client
    global bidOrderId
    global bidOrderDetails
    global wait
    global waitException
    
    if (bidOrderDetails != None):
        time.sleep(round(wait/10))
        while True:
            try:
                bidOrderDetails = client.order_details(bidOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')

#################### Cancel askOrder        

def cancelAsk():
    global client
    global askOrderId
    global askOrderDetails
    global database_past_asks_bids
    global theorySellExecuted
    global theorySellPrice
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    time.sleep(round(wait/10))
    while True:
        try:
            askOrderDetails = client.order_details(askOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    if (askOrderDetails.traded_amount.amount > 0.0):
        theorySellExecuted=theorySellPrice
        write_buy_sell_prices()
    time.sleep(round(wait/10))
    while True:
        try:
            client.cancel_order(askOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    while(True):
        time.sleep(wait)
        while True:
            try:
                askOrderDetails = client.order_details(askOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')
        if (askOrderDetails.state=="canceled") or (askOrderDetails.state=="traded"):
            break
        else:
            time.sleep(wait/2)
            while True:
                try:
                    client.cancel_order(askOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
    
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)

    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
    appen_order_dataframe=pd.DataFrame(columns=columns)
    order_dataframe=pd.DataFrame(askOrderDetails[14]).head(1)
    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
    database_past_asks_bids.reset_index(drop=True,inplace=True)

    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
    pickle.dump(database_past_asks_bids, pickle_out)
    pickle_out.close()
    
    balancing_Ask_Bid()

    #printOutput("Cancel Ask Order [ID]= ",askOrderId)
    askOrderDetails=None
    askOrderId=None

#################### Cancel bidOrder    

def cancelBid():
    global client
    global bidOrderId
    global bidOrderDetails
    global last_volume_traded_fee
    global database_past_asks_bids
    global theoryBuyPrice
    global theoryBuyExecuted
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    time.sleep(round(wait/10))
    while True:
        try:
            bidOrderDetails = client.order_details(bidOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    if (bidOrderDetails.traded_amount.amount > 0.0):
        theoryBuyExecuted=theoryBuyPrice
        last_volume_traded_fee=float(bidOrderDetails.paid_fee[0])+last_volume_traded_fee
        createMarketBidFee()
        write_buy_sell_prices()
    time.sleep(round(wait/10))
    while True:
        try:
            client.cancel_order(bidOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    while(True):
        time.sleep(wait)
        while True:
            try:
                bidOrderDetails = client.order_details(bidOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')
        if (bidOrderDetails.state=="canceled") or (bidOrderDetails.state=="traded"):
            break
        else: 
            time.sleep(wait/2)
            while True:
                try:
                    client.cancel_order(bidOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
    
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)

    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
    appen_order_dataframe=pd.DataFrame(columns=columns)
    order_dataframe=pd.DataFrame(bidOrderDetails[14]).head(1)
    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
    database_past_asks_bids.reset_index(drop=True,inplace=True)

    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
    pickle.dump(database_past_asks_bids, pickle_out)
    pickle_out.close()
    
    balancing_Ask_Bid()
    
    #printOutput("Cancel Bid Order [ID]= ",bidOrderId)
    bidOrderDetails=None
    bidOrderId=None

#################### Cancel bidOrder    

def cancelBidException():
    global client
    global bidOrderId
    global bidOrderDetails
    global last_volume_traded_fee
    global database_past_asks_bids
    global MONEY
    global CRYPT
    global wait
    global waitException
    
    time.sleep(round(wait/10))
    while True:
        try:
            bidOrderDetails = client.order_details(bidOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    time.sleep(round(wait/10))
    while True:
        try:
            client.cancel_order(bidOrderId)
            break
        except:
            time.sleep(waitException)
            #printOutput(f'client error waiting {waitException} seconds')
    while(True):
        time.sleep(wait)
        while True:
            try:
                bidOrderDetails = client.order_details(bidOrderId)
                break
            except:
                time.sleep(waitException)
                #printOutput(f'client error waiting {waitException} seconds')
        if (bidOrderDetails.state=="canceled") or (bidOrderDetails.state=="traded"):
            break
        else:
            time.sleep(wait/2)
            while True:
                try:
                    client.cancel_order(bidOrderId)
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
    
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)

    columns=["id","account_id","amount","created_at","fee_currency","limit","market_id","original_amount","paid_fee","price_type","state","total_exchanged","traded_amount","type","json"]
    appen_order_dataframe=pd.DataFrame(columns=columns)
    order_dataframe=pd.DataFrame(bidOrderDetails[14]).head(1)
    appen_order_dataframe=appen_order_dataframe.append(order_dataframe,sort=False)
    database_past_asks_bids=database_past_asks_bids.append(appen_order_dataframe,sort=False)
    database_past_asks_bids.drop_duplicates(subset=["id"],inplace=True)
    database_past_asks_bids.reset_index(drop=True,inplace=True)

    pickle_out = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","wb")
    pickle.dump(database_past_asks_bids, pickle_out)
    pickle_out.close()
    
    balancing_Ask_Bid()
    
    #printOutput("Cancel Bid Exception Order [ID]= ",bidOrderId)
    bidOrderDetails=None
    bidOrderId=None
    
#################### Get PastAsks and PastBids
    
def updatePast_Asks_Bids():
    
    global pastAsks
    global pastBids
    global database_past_asks_bids
    global MONEY
    global CRYPT
    global MINE
    
    pastAsks=0.0
    pastBids=0.0
    pickle_in = open("database_past_asks_bids_"+MONEY+"_"+CRYPT+".pickle","rb")
    database_past_asks_bids= pickle.load(pickle_in)
    database_past_asks_bids.traded_amount=database_past_asks_bids.traded_amount.astype(float)
    database_past_asks_bids['created_at']= pd.to_datetime(database_past_asks_bids['created_at'])
    
    if MINE!=1:
        database_past_asks_bids=database_past_asks_bids.loc[database_past_asks_bids.created_at>="2020-05-20"]

    pastAsks=database_past_asks_bids.loc[(database_past_asks_bids['type'] == "Ask") &
                                (database_past_asks_bids['price_type'] == "limit") &
                                (database_past_asks_bids["market_id"]==CRYPT+"-"+MONEY) &
                                ((database_past_asks_bids['state'] == "canceled")|
                                 (database_past_asks_bids['state'] == "traded")), 'traded_amount'].sum()

    pastBids=database_past_asks_bids.loc[(database_past_asks_bids['type'] == "Bid") &
                                (database_past_asks_bids['price_type'] == "limit") &
                                (database_past_asks_bids["market_id"]==CRYPT+"-"+MONEY) &
                                ((database_past_asks_bids['state'] == "canceled")|
                                 (database_past_asks_bids['state'] == "traded")), 'traded_amount'].sum()

    pastAsks=round(pastAsks,5)
    pastBids=round(pastBids,5)
    
#################### Balancing Ask & Bid Volumes

def balancing_Ask_Bid():
    
    global pastAsks
    global pastBids
    global askVolume
    global bidVolume
    global askControl
    global bidControl
    global volume

    #printOutput("[Balancing]")
    volume=history_trades()
    controlVariable=abs(pastAsks-pastBids)    
    
    if (pastAsks==pastBids) or (abs(pastAsks-pastBids)<minVolumeTrade):
        askVolume=volume
        bidVolume=volume
    elif pastAsks<pastBids:
        if (round(pastBids-pastAsks,5)>volume):
            subject = "SpreadNet: Stopped (Balancing Problem)"
            msg = "Revise SpreadNet"
            owners = json.loads(config.get('owner_information','OWNERS_ALERT'))
            enviar_alerta(subject, msg, owners)
            #printOutput(errorBalancingAskBid_1)
        else:
            askVolume=round(pastBids-pastAsks,5) if (pastBids-pastAsks>=minVolumeTrade) else 0.0
            bidVolume=0.0
    elif pastAsks>pastBids:
        if (round(pastAsks-pastBids,5)>volume):
            subject = "SpreadNet: Stopped (Balancing Problem)"
            msg = "Revise SpreadNet"
            owners = json.loads(config.get('owner_information','OWNERS_ALERT'))
            enviar_alerta(subject, msg, owners)
            #printOutput(errorBalancingAskBid_2)
        else:
            bidVolume=round(pastAsks-pastBids,5) if (pastAsks-pastBids>=minVolumeTrade) else 0.0
            askVolume=0.0

#################### Write backup from last sell/buy prices
            
def write_buy_sell_prices():
    global theorySellExecuted
    global theoryBuyExecuted
    global last_volume_traded_fee
    global MONEY
    global CRYPT
    last_buy_sell_prices_txt = open("last_buy_sell_prices_"+MONEY+"_"+CRYPT+".txt","w")
    last_buy_sell_prices_dic = {"theorySellExecuted": theorySellExecuted,
                                "theoryBuyExecuted": theoryBuyExecuted,
                                "last_volume_traded_fee":last_volume_traded_fee}
    last_buy_sell_prices_txt.write(str(last_buy_sell_prices_dic))
    last_buy_sell_prices_txt.close()
    

#################### Read backup from last sell/buy prices
    
def read_buy_sell_prices():
    global theorySellExecuted
    global theoryBuyExecuted
    global last_volume_traded_fee
    global MONEY
    global CRYPT
    last_buy_sell_prices_txt = open("last_buy_sell_prices_"+MONEY+"_"+CRYPT+".txt","r")
    last_buy_sell_prices_txt = ast.literal_eval(str(last_buy_sell_prices_txt.read()))
    theoryBuyExecuted=last_buy_sell_prices_txt["theoryBuyExecuted"]
    theorySellExecuted=last_buy_sell_prices_txt["theorySellExecuted"]
    last_volume_traded_fee=last_buy_sell_prices_txt["last_volume_traded_fee"]

#################### Simulate Market Sell
            
def sellAmountACUM(balanceCRY):
    sellAmount=0.0
    sellCounter=0
    sellPriceACUM=0.0
    sellAmountACUM=0.0
    sellingBook=request_order_book()["bids"]
    while sellAmountACUM<balanceCRY:
        sellAmount=float(sellingBook[sellCounter][1])
        sellAmountACUM=sellAmountACUM+sellAmount
        if sellAmountACUM<balanceCRY:
            sellPriceACUM=sellPriceACUM+(sellAmount*float(sellingBook[sellCounter][0]))
            sellCounter=sellCounter+1
        else:
            printCRY=sellAmountACUM-sellAmount+balanceCRY-(sellAmountACUM-sellAmount)
            sellPriceACUM=sellPriceACUM+((balanceCRY-(sellAmountACUM-sellAmount))*float(sellingBook[sellCounter][0]))
    return sellPriceACUM

################### RESET THEORY BUY AND SELL PRICES

def resetTheoryPrices():
    global firstIteration
    global theorySellExecuted
    global theoryBuyExecuted
    global last_volume_traded_fee
    global MONEY
    global CRYPT
    
    theorySellExecuted=updatePrice()[0]
    theoryBuyExecuted=updatePrice()[1]
    last_buy_sell_prices_txt = open("last_buy_sell_prices_"+MONEY+"_"+CRYPT+".txt","r")
    last_buy_sell_prices_txt = ast.literal_eval(str(last_buy_sell_prices_txt.read()))    
    last_volume_traded_fee=last_buy_sell_prices_txt["last_volume_traded_fee"]
    write_buy_sell_prices()
    firstIteration=0
    #printOutput("Reset Theory Prices")
    
#################### Write backup from last sell/buy prices
            
# def #printOutput(*text):
#     global MONEY
#     global CRYPT
#     text='  '.join([str(x) for x in text])
#     with open("output_"+MONEY.lower()+"_"+CRYPT.lower()+".txt","a") as outputTxt:
#         outputTxt.write(text)
#         outputTxt.write("\n")
#         outputTxt.close()
        
#################### REBALANCEO AUTOMÁTICO 1 VEZ AL MES

def monthlyBalance():
    
    global MONEY
    global CRYPT
    global client
    global minVolumeTrade
    global askVolume
    global bidVolume
    global dia_de_balanceo
    global prendido_rebalanceo
    global contador_balanceos
    
    contador_balanceos=1
    
    #_____
    
    prendido_rebalanceo=0
    
    #_____
    
    dia_de_balanceo=min(int(datetime.now().day),27)
    
    #_____
    
    monthlyMoney=getMONinAccount()
    monthlyCrypt=getCRYinAccount()
    monthlyMoneyCrypt=float(pd.DataFrame(client.quotation_market(amount=monthlyCrypt,market_id=CRYPT.lower()+"-"+MONEY.lower(),quotation_type="ask_given_size").json).quote_balance_change[0])

    monthlyTotalMoney=monthlyMoney+monthlyMoneyCrypt
    monthlyBalance=monthlyTotalMoney/2

    if monthlyMoney>monthlyMoneyCrypt:

        monthlyMoneyBuy=monthlyBalance-monthlyMoneyCrypt
        monthlyCryptBuy=float(pd.DataFrame(client.quotation_market(amount=monthlyMoneyBuy,market_id=CRYPT.lower()+"-"+MONEY.lower(),quotation_type="bid_given_value").json).base_balance_change[0])

        if monthlyCryptBuy > minVolumeTrade:

            ### COMPAR MARKET -> monthlyCryptBuy

            while True:
                try:
                    orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "bid", "market", round(monthlyCryptBuy,5))
                    #printOutput("[MarketBid,Balancing]: =",round(monthlyCryptBuy,5))
                    #printOutput("[MarketBid,Balancing]: =")
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')

    else:

        monthlyMoneyCryptSell=monthlyBalance-monthlyMoney
        monthlyCryptSell=float(pd.DataFrame(client.quotation_market(amount=monthlyMoneyCryptSell,market_id=CRYPT.lower()+"-"+MONEY.lower(),quotation_type="bid_given_value").json).base_exchanged[0])

        if monthlyCryptSell > minVolumeTrade:

            ### VENDER MARKET -> monthlyCryptSell

            while True:
                try:
                    orden = client.new_order(CRYPT.lower()+"-"+MONEY.lower(), "ask", "market", round(monthlyCryptSell,5))
                    #printOutput("[MarketAsk,Balancing]: =",round(monthlyCryptSell,5))
                    break
                except:
                    time.sleep(waitException)
                    #printOutput(f'client error waiting {waitException} seconds')
                    
####################
                    
#FUNCIÓN QUE LEE ARCHIVO TXT
def readText(name):
    with open(name) as f:
        lines = f.readlines()
        return lines[0]

# PARAMETERS

In [5]:
updateConfig()
API_KEY = get_config('buda', 'API_KEY')
API_SECRET = get_config('buda', 'API_SECRET')
MINE=float(get_config('buda', 'MINE'))

#################### CLIENT BUDA

client = create_connection_buda()

#################### PARAMETERS
prendido_rebalanceo=1
dia_de_balanceo=0
contador_balanceos=0

commission = float(readText("trade_commission.txt"))
margin = 1/1000
proportion=float(readText("trade_proportion.txt"))

askOrderDetails = None
bidOrderDetails = None
askOrderId = 0
bidOrderId = 0

areMON = True
areCRY = True
balanceCRY = 0.0
balanceMON = 0.0
limitAskPrice = 0.0
limitBidPrice = 0.0
limitAskVolume = 0.0
limitBidVolume = 0.0

theorySellPrice=0.0
theoryBuyPrice=0.0
theorySellExecuted=0.0
theoryBuyExecuted=0.0
last_volume_traded_fee=0.0

pastAsks=0.0
pastBids=0.0
askVolume=0.0
bidVolume=0.0

hourCounter=0

wait=20
waitException=600
porcentaje_reset=0.995

# INITIALIZATION

In [6]:
# crearDatabaseDesdeCero()
# write_buy_sell_prices()
# sys.stdout=open("output_"+CRYPT.lower()+"_"+MONEY.lower()+".txt","w")
# time.sleep(500)

# SAFETY RUN

In [7]:
finishThem_before_run()

# RUN

In [ ]:
###### FIRST ITERATION

firstIteration=0

if firstIteration==1:
    resetTheoryPrices()
else:
    read_buy_sell_prices()

# print_output_file = open("output_"+MONEY+"_"+CRYPT+".txt",'w')
# sys.stdout = print_output_file

###### CONTADOR DE ERROR

contador_error=0
    
###### RUN LIBRARIES

import os

try:
    from IPython.display import clear_output
except:
    pass

#################### UPDATE BALANCE CRYPT + MONEY; UPDATE LIMIT PRICES

getMONinAccount()
getCRYinAccount()
askOrderDetails=None
bidOrderDetails=None
askOrderId=None
bidOrderId=None

#################### 1ST ITERATION

while(True):
    try:
        gotOrder = False
        balancing_Ask_Bid()
        #printOutput("Trades Difference (Asks-Bids)_0: ",round(pastAsks-pastBids,5),datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        while (not gotOrder):
            #printOutput("Sleep 1:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            time.sleep(wait)
            balancing_Ask_Bid()
            updateLimits()
            if (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))) and (askVolume>0.0):
                limitAskPrice -= 1.0
            if (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))) and (bidVolume>0.0):
                limitBidPrice += 1.0
            spread = limitAskPrice - limitBidPrice
            
            if (firstIteration==1) or ((askVolume>0.0) and (bidVolume>0.0)):
                #printOutput('Spread: ',spread,'\t','Margin: ',round((limitAskPrice*(1-2*commission))/limitBidPrice-1,3)*100,'%')
                if (validMargin(limitAskPrice, limitBidPrice)):
                    createAsk(limitAskPrice)
                    createBid(limitBidPrice)
                    #printOutput("Create AskBid : f1")
                    gotOrder = True
                else:
                    #printOutput('No valid margin: wait ',wait,'seconds!')
                    #printOutput("Sleep 2:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    time.sleep(wait)
            else:
                if (askVolume>0.0):
                    if limitAskPrice <= porcentaje_reset*theoryBuyExecuted:
                        #printOutput("Sacrified Margin :",round(abs(limitAskPrice-theoryBuyExecuted)*askVolume,2))
                        resetTheoryPrices()
                    if (validMargin(limitAskPrice, theoryBuyExecuted)):
                        createAsk(limitAskPrice)
                        #printOutput("Create Ask : f1")
                        gotOrder = True
                    else:
                        #printOutput('No valid limitAskPrice: wait ',wait,'seconds!')
                        #printOutput("Sleep 3:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        time.sleep(wait)
                
                if (bidVolume>0.0):
                    if limitBidPrice >= (1+(1-porcentaje_reset))*theorySellExecuted:
                        #printOutput("Sacrified Margin :",round(abs(limitBidPrice-theorySellExecuted)*bidVolume,2))
                        resetTheoryPrices()
                    if (validMargin(theorySellExecuted, limitBidPrice)):
                        createBid(limitBidPrice)
                        #printOutput("Create Bid : f1")
                        gotOrder = True
                    else:
                        #printOutput('No valid limitBidPrice: wait ',wait,'seconds!')
                        #printOutput("Sleep 4:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        time.sleep(wait)

        #################### 2ND ITERATION
        
        while (True):
            
            ########################################################## CHECK IF TRADED

            #printOutput("Sleep 5:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            finishThem()
            time.sleep(wait/10)
            balancing_Ask_Bid()
            time.sleep(wait/10)
            getMONinAccount()
            time.sleep(wait/10)
            getCRYinAccount()

            #printOutput("[Checking Traded] :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            if (askOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        askOrderDetails = client.order_details(askOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
            else:
                askOrderDetails = None
            if (bidOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        bidOrderDetails = client.order_details(bidOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
            else:
                bidOrderDetails = None
                
            if (askOrderDetails != None):
                gotNewAsk = (askOrderDetails.traded_amount.amount > 0.0)
                if (gotNewAsk):
                    theorySellExecuted=theorySellPrice
                    write_buy_sell_prices()
                    cancelAsk()
            else:
                gotNewAsk=False

            if (bidOrderDetails != None):
                gotNewBid = (bidOrderDetails.traded_amount.amount > 0.0)
                if (gotNewBid):
                    theoryBuyExecuted=theoryBuyPrice                        
                    write_buy_sell_prices()
                    cancelBid()
            else:
                gotNewBid=False

            if (gotNewAsk) or (gotNewBid):

                #printOutput("Sleep 6:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                time.sleep(wait/2)
                balancing_Ask_Bid()
                #printOutput("Trades Difference (Asks-Bids)_1: ",round(pastAsks-pastBids,5),datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                #printOutput("[askVolume,bidVolume] :",askVolume,bidVolume)

                if (askVolume==0.0) and (bidVolume>0.0):
                    testigo_bid=1
                    while testigo_bid==1:
                        limitBidPrice = updatePrice()[1]
                        limitBidVolume= updateVolume()[1]
                        read_buy_sell_prices()
                        if (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))):
                            limitBidPrice += 1.0
                        if limitBidPrice >= (1+(1-porcentaje_reset))*theorySellExecuted:
                            #printOutput("Sacrified Margin :",round(abs(limitBidPrice-theorySellExecuted)*bidVolume,2))
                            resetTheoryPrices()
                        if (validMargin(theorySellExecuted,limitBidPrice)) and (bidOrderDetails==None):
                            createBid(limitBidPrice)
                            testigo_bid=0
                            #printOutput("Create Bid : 0")
                        else:
                            if bidOrderDetails!= None:
                                cancelBid()
                            #printOutput("Sleep 7:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                            time.sleep(wait/2)

                balancing_Ask_Bid()
                
                if (bidVolume==0.0) and (askVolume>0.0):
                    testigo_ask=1
                    while testigo_ask==1:
                        limitAskPrice = updatePrice()[0]
                        limitAskVolume= updateVolume()[0]
                        read_buy_sell_prices()
                        if (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))):
                            limitAskPrice -= 1.0
                        if limitAskPrice <= porcentaje_reset*theoryBuyExecuted:
                            #printOutput("Sacrified Margin :",round(abs(limitAskPrice-theoryBuyExecuted)*askVolume,2))
                            resetTheoryPrices()
                        if (validMargin(limitAskPrice,theoryBuyExecuted)) and (askOrderDetails==None):
                            createAsk(limitAskPrice)
                            testigo_ask=0
                            #printOutput("Create Ask : 0")
                        else:
                            if askOrderDetails!= None:
                                cancelAsk()
                      
                            #printOutput("Sleep 8:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                            time.sleep(wait/2)
                            
                balancing_Ask_Bid()

                if (bidVolume>0.0) and (askVolume>0.0):
                    testigo_ask_bid=1
                    while testigo_ask_bid==1:
                        updateLimits()
                        if (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))):
                            limitAskPrice -= 1.0
                        if (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))):
                            limitBidPrice += 1.0
                        if (validMargin(limitAskPrice,limitBidPrice)):
                            createAsk(limitAskPrice)
                            createBid(limitBidPrice)
                            testigo_ask_bid=0
                            #printOutput("Create Ask_Bid : 0")
                        else:
                            if askOrderDetails!= None:
                                cancelAsk()
                            if bidOrderDetails!= None:
                                cancelBid()
                                
                            #printOutput("Sleep 9:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                            time.sleep(wait/2)

            
            ########################################################## CHECK MARKET CHANGES
            #printOutput("Sleep 10:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            time.sleep(wait/2)
            finishThem()
            
            if (askOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        askOrderDetails = client.order_details(askOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
                gotNewAsk_Traded = (askOrderDetails.traded_amount.amount > 0.0)
            else:
                askOrderDetails = None
                gotNewAsk_Traded = False
            if (bidOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        bidOrderDetails = client.order_details(bidOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
                gotNewBid_Traded = (bidOrderDetails.traded_amount.amount > 0.0)
            else:
                bidOrderDetails = None
                gotNewBid_Traded = False
                
            #printOutput("[Checking Traded Before Market]",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            if not(gotNewAsk_Traded) and not(gotNewBid_Traded):
                #printOutput("[Checking Market] :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                newAskPrice, newBidPrice = updatePrice()
                balancing_Ask_Bid()
                
                #printOutput("Trades Difference (Asks-Bids)_3: ",round(pastAsks-pastBids,5),datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                newLimitAskVolume= updateVolume()[0]
                newLimitBidVolume= updateVolume()[1]
                
                if (askOrderDetails != None) and (bidOrderDetails == None):                                     
                    gotNewAsk = True if (newAskPrice != theorySellPrice) or ((theorySellPrice+1!=float(request_order_book()["asks"][1][0])) and (askVolume==newLimitAskVolume) and (newAskPrice == theorySellPrice)) or (float(askVolume/(newLimitAskVolume))<float(readText("trade_proportion.txt")) and (newAskPrice == theorySellPrice)) else False
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            askOrderDetails = client.order_details(askOrderId)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (gotNewAsk) and (askVolume>0.0) and (bidVolume==0.0) and (askOrderDetails.traded_amount.amount == 0.0):
                        cancelAsk()
                        testigo_ask=1
                        while testigo_ask==1:
                            #printOutput("Sleep 11:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                            time.sleep(wait/2)
                            limitAskPrice = updatePrice()[0]
                            limitAskVolume= updateVolume()[0]
                            read_buy_sell_prices()
                            if (round(askVolume,5)>0.0) and (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))):
                                limitAskPrice -= 1.0
                            if limitAskPrice <= porcentaje_reset*theoryBuyExecuted:
                                #printOutput("Sacrified Margin :",round(abs(limitAskPrice-theoryBuyExecuted)*askVolume,2))
                                resetTheoryPrices()
                            if (validMargin(limitAskPrice,theoryBuyExecuted)) and (askOrderDetails==None):
                                createAsk(limitAskPrice)
                                testigo_ask=0
                                #printOutput("Create Ask : 1")
                            else:
                                #printOutput('No valid limitAskPrice: wait ',wait/2,'seconds!')
                                if askOrderDetails!= None:
                                    cancelAsk()
                    
                    elif (askOrderDetails.traded_amount.amount > 0.0):
                        cancelAsk()
                
                if (bidOrderDetails != None) and (askOrderDetails == None):
                    gotNewBid = True if (newBidPrice != theoryBuyPrice) or ((theoryBuyPrice-1!=float(request_order_book()["bids"][1][0])) and (bidVolume==newLimitBidVolume) and (newBidPrice == theoryBuyPrice)) or (float(bidVolume/(newLimitBidVolume))<float(readText("trade_proportion.txt")) and (newBidPrice == theoryBuyPrice)) else False
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            bidOrderDetails = client.order_details(bidOrderId)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (gotNewBid) and (bidVolume>0.0) and (askVolume==0.0) and (bidOrderDetails.traded_amount.amount == 0.0):
                        cancelBid()
                        testigo_bid=1
                        while testigo_bid==1:
                            limitBidPrice = updatePrice()[1]
                            limitBidVolume= updateVolume()[1]
                            read_buy_sell_prices()
                            if (round(bidVolume,5)>0.0) and (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))):
                                limitBidPrice += 1.0
                            if limitBidPrice >= (1+(1-porcentaje_reset))*theorySellExecuted:
                                #printOutput("Sacrified Margin :",round(abs(limitBidPrice-theorySellExecuted)*bidVolume,2))
                                resetTheoryPrices()
                            if (validMargin(theorySellExecuted,limitBidPrice)) and (bidOrderDetails==None):
                                createBid(limitBidPrice)
                                testigo_bid=0
                                #printOutput("Create Bid : 1")
                            else:
                                #printOutput("Sleep 12:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                                #printOutput('No valid limitBidPrice: wait ',wait/2,'seconds!')
                                time.sleep(wait/2)
                                if bidOrderDetails!= None:
                                    cancelBid()
                                
                    elif (bidOrderDetails.traded_amount.amount > 0.0):
                        cancelBid()

                if (askOrderDetails != None) and (bidOrderDetails != None):
                    gotNewAsk = True if (newAskPrice != theorySellPrice) or ((theorySellPrice+1!=float(request_order_book()["asks"][1][0])) and (askVolume==newLimitAskVolume) and (newAskPrice == theorySellPrice)) or (float(askVolume/(newLimitAskVolume))<float(readText("trade_proportion.txt")) and (newAskPrice == theorySellPrice)) else False
                    gotNewBid = True if (newBidPrice != theoryBuyPrice) or ((theoryBuyPrice-1!=float(request_order_book()["bids"][1][0])) and (bidVolume==newLimitBidVolume) and (newBidPrice == theoryBuyPrice)) or (float(bidVolume/(newLimitBidVolume))<float(readText("trade_proportion.txt")) and (newBidPrice == theoryBuyPrice)) else False
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            askOrderDetails = client.order_details(askOrderId)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    time.sleep(round(wait/10))
                    while True:
                        try:
                            bidOrderDetails = client.order_details(bidOrderId)
                            break
                        except:
                            time.sleep(waitException)
                            #printOutput(f'client error waiting {waitException} seconds')
                    if (askOrderDetails.traded_amount.amount == 0.0):
                        if (gotNewAsk) and (askVolume>0.0) and (bidVolume>0.0):
                            cancelAsk()
                            limitAskPrice = updatePrice()[0]
                            limitAskVolume= updateVolume()[0]
                            if (round(askVolume,5)>0.0) and (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))):
                                limitAskPrice -= 1.0
                        else:
                            limitAskPrice = theorySellPrice
                    else:
                        cancelAsk()
                    if (bidOrderDetails.traded_amount.amount == 0.0):
                        if (gotNewBid) and (bidVolume>0.0) and (askVolume>0.0):
                            cancelBid()
                            limitBidPrice = updatePrice()[1]
                            limitBidVolume= updateVolume()[1]
                            if (round(bidVolume,5)>0.0) and (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))):
                                limitBidPrice += 1.0
                        else:
                            limitBidPrice = theoryBuyPrice
                    else:
                        cancelBid()
                    balancing_Ask_Bid()
                    if (askOrderId!=None):
                        time.sleep(round(wait/10))
                        while True:
                            try:
                                askOrderDetails = client.order_details(askOrderId)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                    else:
                        askOrderDetails = None
                    if (bidOrderId!=None):
                        time.sleep(round(wait/10))
                        while True:
                            try:
                                bidOrderDetails = client.order_details(bidOrderId)
                                break
                            except:
                                time.sleep(waitException)
                                #printOutput(f'client error waiting {waitException} seconds')
                    else:
                        bidOrderDetails = None
                    if (askOrderDetails == None) and (askVolume>0.0) and (bidOrderDetails == None) and (bidVolume>0.0):
                        testigo_ask_bid=1
                        while testigo_ask_bid==1:
                            if (validMargin(limitAskPrice,limitBidPrice)):
                                createAsk(limitAskPrice)
                                createBid(limitBidPrice)
                                testigo_ask_bid=0
                                #printOutput("Create Ask_Bid : 1")
                            else:
                                #printOutput("Sleep 13:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                                #printOutput('No valid limitAskBidPrice: wait ',wait/2,'seconds!')
                                time.sleep(wait/2)
                                updateLimits()
                                if (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))):
                                    limitAskPrice -= 1.0
                                if (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))):
                                    limitBidPrice += 1.0

                    balancing_Ask_Bid()
                    if (askOrderDetails == None) and (askVolume>0.0) and (bidOrderDetails != None) and (bidVolume>0.0):
                        if (bidOrderDetails.traded_amount.amount == 0.0):
                            if (validMargin(limitAskPrice,limitBidPrice)):
                                createAsk(limitAskPrice)
                                #printOutput("Create Ask : 2")
                            else:
                                cancelBid()
                    balancing_Ask_Bid()
                    if (askOrderDetails != None) and (askVolume>0.0) and (bidOrderDetails == None) and (bidVolume>0.0):
                        if (askOrderDetails.traded_amount.amount == 0.0):
                            if (validMargin(limitAskPrice,limitBidPrice)):
                                createBid(limitBidPrice)
                                #printOutput("Create Bid : 2")
                            else:
                                cancelAsk()

            ########################################################## RE-LOOP
            
            #printOutput("Sleep 14:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            time.sleep(wait/2)
            finishThem()
            
            if (askOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        askOrderDetails = client.order_details(askOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
            else:
                askOrderDetails=None
            if (bidOrderId!=None):
                time.sleep(round(wait/10))
                while True:
                    try:
                        bidOrderDetails = client.order_details(bidOrderId)
                        break
                    except:
                        time.sleep(waitException)
                        #printOutput(f'client error waiting {waitException} seconds')
            else:
                bidOrderDetails=None
            
            #printOutput("[Checking Reloop] :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            if (askOrderDetails == None) and (bidOrderDetails == None):
                gotOrder = False
                balancing_Ask_Bid()
                while (not gotOrder):
                    updateLimits()
                    #printOutput("Trades Difference (Asks-Bids)_4: ",round(pastAsks-pastBids,5),datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    if (float(askVolume/(limitAskVolume+askVolume))<float(readText("trade_proportion.txt"))) and (askVolume>0.0):
                        limitAskPrice -= 1.0
                    if (float(bidVolume/(limitBidVolume+bidVolume))<float(readText("trade_proportion.txt"))) and (bidVolume>0.0):
                        limitBidPrice += 1.0
                    spread = limitAskPrice - limitBidPrice
                    #printOutput('Spread: ',spread,'\t','Margin: ',round((limitAskPrice*(1-2*commission))/limitBidPrice-1,3)*100,'%')
                    if (validMargin(limitAskPrice, limitBidPrice)):
                        createAsk(limitAskPrice)
                        createBid(limitBidPrice)
                        gotOrder = True
                        #printOutput("Create Ask_Bid : 2")
                    else:
                        #printOutput('No valid margin: wait', wait,'seconds!')
                        #printOutput("Sleep 15:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        time.sleep(wait)
                        
            contador_error=0
            
    except Exception as inst:
        
        while (askOrderId!=None) or (bidOrderId!=None):
            try:
                finishThem_before_run()
            except:
                pass
                time.sleep(wait/2)
        
        exc_type, exc_obj, tb = sys.exc_info()
        lineno = tb.tb_lineno
        
        #_____
        
        string_not_error_1="line 1 column 1 (char 0)"
        
        #_____
        
        if (str(exc_type)=="<class 'requests.exceptions.ConnectionError'>") or (str(exc_type)=="<class 'trading_api_wrappers.errors.InvalidResponse'>"):
            #printOutput(" ---- Exception 1 {} in line {}".format(inst,lineno))
            time.sleep(wait*60)
            #printOutput("[askOrderId, bidOrderId]:",askOrderId,bidOrderId)
        else:
            contador_error=contador_error+1
            exc_type, exc_obj, tb = sys.exc_info()
            lineno = tb.tb_lineno
            #printOutput(" ---- Exception 2 {} in line {}".format(inst,lineno))
            if (contador_error>5) and (string_not_error_1 not in str(inst)):
                subject = "SpreadNet: Stopped (Exception) - ("+CRYPT+"/"+MONEY+")"
                msg = "Stopped<br><br><b>Exception:</b><br><br>{}<br><br>Line:{}".format(inst,lineno)
                owners = json.loads(config.get('owner_information','OWNERS_ALERT'))
                enviar_alerta(subject, msg, owners)
                contador_error=0
            else:
                time.sleep(wait*120)

# FINISH

In [ ]:
finishThem()

_______________________________________________________

# PSEUDOCÓDIGO

In [ ]:
#_____LOOP INFINITO

    #_____LOOP PRIMERA ITERACIÓN
    
        #_____CANCELAR CUALQUIER TIPO DE ORDEN EXISTENTE + BALANCEO # 1

        #_____SI NO TENGO NINGUNA ORDEN MONTADA
        
            #####
        
            #_____SI ESTOY EN ESCENARIO ASK
            
                #_____LOOP HASTA QUE MONTE LA TRANSACCIÓN
            
                    #_____ACTUALIZAR LIBRO DE ORDENES # 2

                    #_____DETERMIAR EL PRECIO TEÓRICO TRANSACCIÓN # A

                    #_____DETERMINAR SI HAY SPREAD # B
                
                    #_____SI TENGO SPREAD
                    
                        #_____COLOCO POSICIÓN LIMIT ASK # 3
                        
                        #_____SALGO DEL LOOP PRIMERA ITERACIÓN
                        
                    #_____SI NO TENGO SPREAD
                    
                        #_____ESPERAR
                        
            #####
                    
            #_____ELIF Y SI ESTOY EN ESCENARIO BID
            
                #_____ACTUALIZAR LIBRO DE ORDENES
                
                #_____DETERMIAR EL PRECIO TEÓRICO TRANSACCIÓN # A
                
                #_____DETERMINAR SI HAY SPREAD # B
                
                #_____SI TENGO SPREAD
                    
                    #_____COLOCO POSICIÓN LIMIT BID # 4
                    
                    #_____SALGO DEL LOOP PRIMERA ITERACIÓN
                        
                #_____SI NO TENGO SPREAD
                    
                    #_____ESPERAR
            
            #####
            
            #_____ELIF Y SI ESTOY EN ESCENARIO ASK + BID
            
                #_____ACTUALIZAR LIBRO DE ORDENES
                
                #_____DETERMIAR EL PRECIO TEÓRICO TRANSACCIÓN # A
                
                #_____DETERMINAR SI HAY SPREAD # B
                
                #_____SI TENGO SPREAD
                    
                    #_____COLOCO POSICIÓN LIMIT ASK # 3
                    
                    #_____COLOCO POSICIÓN LIMIT BID # 4
                    
                    #_____SALGO DEL LOOP PRIMERA ITERACIÓN
                        
                #_____SI NO TENGO SPREAD
                    
                    #_____ESPERAR
        

In [ ]:
# 1: SI TENGO ORDEN + CANCELO + ACTUALIZO BASE DE DATOS + BALANCEO

# 2: ACTUALIZAR VARIABLE GLOBAL DE PRECIOS PUNTA DEL LIBRO DE ORDENES

# 3: COLOCA POSICIÓN ASK + PRECIO + DETALLE TRANSACCIÓN + ID TRANSACCIÓN + TESTIGO DE QUE TENGO ORDEN ASK

# 4: COLOCA POSICIÓN BID + PRECIO + DETALLE TRANSACCIÓN + ID TRANSACCIÓN + TESTIGO DE QUE TENGO ORDEN BID

In [ ]:
# A: VALOR QUE ME DICE CUÁNTO ME DEBO ALEJAR DEL SPREAD ($)

# B: VALOR DEL SPREAD MÍNIMO AL CUAL DEBERÍA OPERAR

In [5]:
import pickle

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):
    while True:
        try:
            return pickle.load(open(variable_name+".pickle","rb"))
            break
        except:
            time.sleep(1)
            

payOutsDataFrame=readPickleVariable("/home/ubuntu/Circle/payOutsDataFrame")
payOutsDataFrame.to_csv("payOutsDataFrame.csv")